In [27]:
import numpy as np
import pandas as pd
from sklearn import svm
import random
import copy
import matplotlib.pyplot as plt
import math
from sklearn.utils import shuffle

In [366]:
class GAIndividual:
 
    '''
    individual of genetic algorithm
    '''
 
    def __init__(self,  vardim, bound):
        '''
        vardim: dimension of variables
        bound: boundaries of variables
        '''
        self.vardim = vardim
        self.bound = bound
        self.fitness = 0.
 
    def generate(self):
        '''
        generate a random chromsome for genetic algorithm
        '''
        len = self.vardim
        rnd = np.random.random(size=len)
        self.chrom = np.zeros(len)
        for i in range(0, len):
            self.chrom[i] = self.bound[0, i] + \
                (self.bound[1, i] - self.bound[0, i]) * rnd[i]
 
    def calculateFitness(self,dataset):
        '''
        calculate the fitness of the chromsome
        '''
        #print(self.chrom)
        self.fitness = SVMResult(
            self.vardim, self.chrom, self.bound, dataset)
        
    def print_(self):
        print(self.chrom)

In [367]:
class GA:
 
    '''
    The class for genetic algorithm
    '''
 
    def __init__(self, sizepop, vardim, bound, MAXGEN, params):
        '''
        sizepop: population sizepop
        vardim: dimension of variables
        bound: boundaries of variables
        MAXGEN: termination condition
        param: algorithm required parameters, it is a list which is consisting of crossover rate, mutation rate, alpha
        '''
        self.sizepop = sizepop
        self.MAXGEN = MAXGEN
        self.vardim = vardim
        self.bound = bound
        self.population = []
        self.fitness = np.zeros((self.sizepop, 1))
        self.trace = np.zeros((self.MAXGEN, 2))
        self.params = params
        self.dataset = creat_dataset(pd.read_csv("diabetes.csv"))
        
    def creat_dataset(df):
        df = shuffle(df)
        size = len(df)
        df['split'] = 0
        df.iloc[0:math.ceil(0.7*size),-1] = 'train'
        df.iloc[math.ceil(0.7*size):math.ceil(0.85*size), -1] = 'test'
        df.iloc[math.ceil(0.85*size):size, -1] = 'val'

        return df
 
    def initialize(self):
        '''
        initialize the population
        '''
        for i in range(0, self.sizepop):
            ind = GAIndividual(self.vardim, self.bound)
            ind.generate()
            self.population.append(ind)
 
    def evaluate(self):
        '''
        evaluation of the population fitnesses
        '''
        for i in range(0, self.sizepop):
            self.population[i].calculateFitness(self.dataset)
            #print("###/n")
            self.fitness[i] = self.population[i].fitness
    
    def show(self):
        for i in range(0, self.sizepop):
            print('chorme {}: '.format(i))
            self.population[i].print_()
 
    def solve(self):
        '''
        evolution process of genetic algorithm
        '''
        self.t = 0
        self.initialize()
        self.evaluate()
        best = np.max(self.fitness)
        bestIndex = np.argmax(self.fitness)
        self.best = copy.deepcopy(self.population[bestIndex])
        self.avefitness = np.mean(self.fitness)
        self.trace[self.t, 0] = self.best.fitness
        self.trace[self.t, 1] = self.avefitness
        #self.trace[self.t, 0] = (1 - self.best.fitness) / self.best.fitness
        #self.trace[self.t, 1] = (1 - self.avefitness) / self.avefitness
        print("Generation %d: optimal function value is: %f; average function value is %f" % (
            self.t, self.trace[self.t, 0], self.trace[self.t, 1]))
        while (self.t < self.MAXGEN - 1):
            self.t += 1
            self.selection()
            self.crossover()
            self.mutation()
            self.evaluate()
            best = np.max(self.fitness)
            bestIndex = np.argmax(self.fitness)
            if best > self.best.fitness:
                self.best = copy.deepcopy(self.population[bestIndex])
            self.avefitness = np.mean(self.fitness)
            self.trace[self.t, 0] = self.best.fitness
            self.trace[self.t, 1] = self.avefitness
            #self.trace[self.t, 0] = (1 - self.best.fitness) / self.best.fitness
            #self.trace[self.t, 1] = (1 - self.avefitness) / self.avefitness
            print("Generation %d: optimal function value is: %f; average function value is %f" % (
                self.t, self.trace[self.t, 0], self.trace[self.t, 1]))
 
        print("Optimal function value is: %f; " %
              self.trace[self.t, 0])
        print("Optimal solution is:")
        print(self.best.chrom)
        self.printResult()
 
    def selection(self):
        new_pop = []
        #print("max: {}".format(np.argmax(self.fitness)))
        p = np.exp(self.fitness*20)/np.sum(np.exp(self.fitness*20))#调大
        #print(p)
        sum = 0
        for i in range(0, self.sizepop): #构造概率空间
            p[i] = p[i] + sum
            sum = p[i]
        p = np.append(p,0)
        for i in range(0, self.sizepop):
            r = random.random()
            for i in range(0, self.sizepop):
                if p[i-1]<=r and p[i]>=r:
                    #print(i)
                    new_pop.append(self.population[i])
                    break
        self.population = new_pop
 
    def crossover(self):
        newpop = []
        for i in range(0, self.sizepop, 2):
            idx1 = random.randint(0, self.sizepop - 1)
            idx2 = random.randint(0, self.sizepop - 1)
            while idx2 == idx1:
                idx2 = random.randint(0, self.sizepop - 1)
            newpop.append(copy.deepcopy(self.population[idx1]))
            newpop.append(copy.deepcopy(self.population[idx2]))
            r = random.random()
            if r < self.params[0]:
                Pos = random.randint(1, self.vardim - 1)
                for j in range(Pos, self.vardim):#部分交换
                    newpop[i].chrom[j] = newpop[i].chrom[j] * self.params[2] + (1 - self.params[2]) * newpop[i + 1].chrom[j]
                    newpop[i + 1].chrom[j] = newpop[i + 1].chrom[j] * self.params[2] + (1 - self.params[2]) * newpop[i].chrom[j]
        self.population = newpop 
 
    def mutation(self):
        
        newpop = []
        for i in range(0, self.sizepop):
            newpop.append(copy.deepcopy(self.population[i]))
            
            for j in range(0, self.vardim):
                r = random.random()
                if r < self.params[1]:
                    pos = random.randint(0, self.vardim - 1)
                    theta = random.random()
                    if theta > 0.5:
                        newpop[i].chrom[pos] = newpop[i].chrom[pos] + (newpop[i].chrom[pos] - self.bound[0,pos]) * (1 - random.random())
                    else:
                        newpop[i].chrom[pos] = newpop[i].chrom[pos] + (self.bound[1,pos] - newpop[i].chrom[pos]) * (1 - random.random())
                        #** (1 - self.t / self.MAXGEN)
        self.population = newpop
 
    def printResult(self):
        '''
        plot the result of the genetic algorithm
        '''
        x = np.arange(0, self.MAXGEN)
        y1 = self.trace[:, 0]
        y2 = self.trace[:, 1]
        plt.plot(x, y1, 'r', label='optimal value')
        plt.plot(x, y2, 'g', label='average value')
        plt.xlabel("Iteration")
        plt.ylabel("function value")
        plt.title("Genetic algorithm for function optimization")
        plt.legend()
        plt.show()


In [368]:
def msefunc(predictval,realval):
    Error = []
    for i in range(len(predictval)):
        val=predictval[i-1]-realval[i-1]
        Error.append(val)  # target-prediction之差平方 
 
   # print("Square Error: ", Error)
    #print("E = ", sum(Error) / len(Error))  # 均方误差MSE
    return sum(Error) / len(Error)
 
def SVMResult(vardim, x, bound, dataset):
    X = dataset.loc[dataset['split'] == 'train'].iloc[:,0:-2].values
    y = dataset.loc[dataset['split'] == 'train'].iloc[:,-2].values
    val_X = dataset.loc[dataset['split'] == 'val'].iloc[:,0:-2].values
    val_y = dataset.loc[dataset['split'] == 'val'].iloc[:,-2].values
    c = x[0]
    g = x[1]  #四参数
    clf = svm.SVC(C=c, gamma=g, kernel='rbf')
    clf.fit(X, y)
    #predictval=clf.predict(val_X)
    #print(predictval)
    #return msefunc(predictval,val_y) 
    return clf.score(val_X, val_y)


In [371]:
bound = np.array([[0,0],[10,0.0001]])
ga = GA(40, 2, bound, 1000, [0.5, 0.1, 0.5])#调低种群大小
ga.solve()

W:\conda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
W:\conda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Generation 0: optimal function value is: 0.765217; average function value is 0.733478
Generation 1: optimal function value is: 0.773913; average function value is 0.743261
Generation 2: optimal function value is: 0.773913; average function value is 0.745217
Generation 3: optimal function value is: 0.773913; average function value is 0.752609
Generation 4: optimal function value is: 0.773913; average function value is 0.754565
Generation 5: optimal function value is: 0.773913; average function value is 0.752391
Generation 6: optimal function value is: 0.773913; average function value is 0.757174
Generation 7: optimal function value is: 0.773913; average function value is 0.756087
Generation 8: optimal function value is: 0.773913; average function value is 0.758696
Generation 9: optimal function value is: 0.773913; average function value is 0.752826
Generation 10: optimal function value is: 0.773913; average function value is 0.753478
Generation 11: optimal function value is: 0.773913; a

Generation 95: optimal function value is: 0.773913; average function value is 0.756522
Generation 96: optimal function value is: 0.773913; average function value is 0.754565
Generation 97: optimal function value is: 0.773913; average function value is 0.751957
Generation 98: optimal function value is: 0.773913; average function value is 0.749130
Generation 99: optimal function value is: 0.773913; average function value is 0.753261
Generation 100: optimal function value is: 0.773913; average function value is 0.756739
Generation 101: optimal function value is: 0.773913; average function value is 0.753913
Generation 102: optimal function value is: 0.773913; average function value is 0.751957
Generation 103: optimal function value is: 0.773913; average function value is 0.748696
Generation 104: optimal function value is: 0.773913; average function value is 0.750435
Generation 105: optimal function value is: 0.773913; average function value is 0.746739
Generation 106: optimal function valu

Generation 189: optimal function value is: 0.773913; average function value is 0.727609
Generation 190: optimal function value is: 0.773913; average function value is 0.728261
Generation 191: optimal function value is: 0.773913; average function value is 0.727609
Generation 192: optimal function value is: 0.773913; average function value is 0.725652
Generation 193: optimal function value is: 0.773913; average function value is 0.726304
Generation 194: optimal function value is: 0.773913; average function value is 0.727391
Generation 195: optimal function value is: 0.773913; average function value is 0.727609
Generation 196: optimal function value is: 0.773913; average function value is 0.729130
Generation 197: optimal function value is: 0.773913; average function value is 0.729783
Generation 198: optimal function value is: 0.773913; average function value is 0.729565
Generation 199: optimal function value is: 0.773913; average function value is 0.728696
Generation 200: optimal function

Generation 283: optimal function value is: 0.773913; average function value is 0.720000
Generation 284: optimal function value is: 0.773913; average function value is 0.722826
Generation 285: optimal function value is: 0.773913; average function value is 0.725870
Generation 286: optimal function value is: 0.773913; average function value is 0.723696
Generation 287: optimal function value is: 0.773913; average function value is 0.721522
Generation 288: optimal function value is: 0.773913; average function value is 0.721522
Generation 289: optimal function value is: 0.773913; average function value is 0.724130
Generation 290: optimal function value is: 0.773913; average function value is 0.728478
Generation 291: optimal function value is: 0.773913; average function value is 0.731087
Generation 292: optimal function value is: 0.773913; average function value is 0.731304
Generation 293: optimal function value is: 0.773913; average function value is 0.729565
Generation 294: optimal function

Generation 377: optimal function value is: 0.773913; average function value is 0.730217
Generation 378: optimal function value is: 0.773913; average function value is 0.730217
Generation 379: optimal function value is: 0.773913; average function value is 0.730000
Generation 380: optimal function value is: 0.773913; average function value is 0.729783
Generation 381: optimal function value is: 0.773913; average function value is 0.730652
Generation 382: optimal function value is: 0.773913; average function value is 0.731087
Generation 383: optimal function value is: 0.773913; average function value is 0.729565
Generation 384: optimal function value is: 0.773913; average function value is 0.729348
Generation 385: optimal function value is: 0.773913; average function value is 0.728261
Generation 386: optimal function value is: 0.773913; average function value is 0.728043
Generation 387: optimal function value is: 0.773913; average function value is 0.727174
Generation 388: optimal function

Generation 471: optimal function value is: 0.773913; average function value is 0.731957
Generation 472: optimal function value is: 0.773913; average function value is 0.737609
Generation 473: optimal function value is: 0.773913; average function value is 0.735000
Generation 474: optimal function value is: 0.773913; average function value is 0.735435
Generation 475: optimal function value is: 0.773913; average function value is 0.725870
Generation 476: optimal function value is: 0.773913; average function value is 0.727609
Generation 477: optimal function value is: 0.773913; average function value is 0.731739
Generation 478: optimal function value is: 0.773913; average function value is 0.732391
Generation 479: optimal function value is: 0.773913; average function value is 0.734783
Generation 480: optimal function value is: 0.773913; average function value is 0.733043
Generation 481: optimal function value is: 0.773913; average function value is 0.732174
Generation 482: optimal function

KeyboardInterrupt: 

In [ ]:
def creat_dataset(df):
        df = shuffle(df)
        size = len(df)
        df['split'] = 0
        df.iloc[0:math.ceil(0.7*size),-1] = 'train'
        df.iloc[math.ceil(0.7*size):math.ceil(0.85*size), -1] = 'test'
        df.iloc[math.ceil(0.85*size):size, -1] = 'val'

        return df
dataset = creat_dataset(pd.read_csv("diabetes.csv"))

In [139]:
bound = np.tile([[0], [1]], 2) 
bound

array([[0, 0],
       [1, 1]])

In [355]:
def SVMResult(dataset):
    X = dataset.loc[dataset['split'] == 'train'].iloc[:,0:-2].values
    y = dataset.loc[dataset['split'] == 'train'].iloc[:,-2].values
    val_X = dataset.loc[dataset['split'] == 'val'].iloc[:,0:-2].values
    val_y = dataset.loc[dataset['split'] == 'val'].iloc[:,-2].values
    
    clf = svm.SVC(C=1, gamma=0.00003, kernel='rbf')
    clf.fit(X, y)
    #predictval=clf.predict(val_X)
    #print(predictval)
    #return msefunc(predictval,val_y) 
    return clf.score(val_X, val_y)
print(SVMResult(dataset))

0.7913043478260869
